Librerias a utilizar

In [351]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score

In [352]:
df = pd.read_csv('train titanic.csv')

## Procesamiento de datos

In [353]:
df.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [354]:
df

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


Dividimos la columna 'Cabin', en 3 columnas

In [355]:
df[['Deck', 'Num', 'Side']] = df.Cabin.str.split('/', expand=True)

In [356]:
df = df[['PassengerId',	'HomePlanet',	'CryoSleep', 'Deck', 'Num', 'Side', 'Cabin', 'Destination',	'Age',	'VIP',
         'RoomService',	'FoodCourt',	'ShoppingMall',	'Spa',	'VRDeck',	'Name',	'Transported']]

In [357]:
df['Num'] = pd.to_numeric(df['Num'], errors='coerce')

In [358]:
df.describe()

,Num,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8494.000000,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000
mean,600.367671,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791
std,511.867226,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,167.250000,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,427.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,999.000000,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000
max,1894.000000,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


Identificamos Outliers, así como eliminarlos, esto con el proposito de no afectar la manera de llenado en las columnas numericas, en esto caso, utilizando el promedio

In [359]:
num_columns = df.describe().columns
for column in num_columns:
  print(column)
  up =df[column].mean() + 3*df[column].std()
  lw =df[column].mean() - 3*df[column].std()
  index = df[(df[column]>up) | (df[column]<lw)].index
  df.drop(index, inplace=True)

Num
Age
RoomService
FoodCourt
ShoppingMall
Spa
VRDeck


In [360]:
df.drop(['PassengerId', 'Cabin', 'Name'], inplace=True, axis='columns')

In [361]:
df.describe()

,Num,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,7731.000000,7741.000000,7739.000000,7735.000000,7716.000000,7738.000000,7728.000000
mean,629.484155,28.176592,157.539217,244.199871,122.848367,152.817524,149.440994
std,514.289641,14.404563,374.027357,725.911893,308.401503,419.689996,415.251073
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,185.000000,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,478.000000,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1034.500000,37.000000,33.000000,19.000000,17.000000,22.000000,14.000000
max,1894.000000,72.000000,2215.000000,5252.000000,1973.000000,3208.000000,3291.000000


Llenamos los valores faltantes númericos con el valor promedio

In [362]:
num_imput = SimpleImputer(missing_values=np.NaN, strategy='mean')
df[num_columns] = num_imput.fit_transform(df[num_columns])

In [363]:
df[num_columns] = StandardScaler().fit_transform(df[num_columns])

In [364]:
df.describe()

,Num,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,7.906000e+03,7.906000e+03,7.906000e+03,7.906000e+03,7.906000e+03,7.906000e+03,7.906000e+03
mean,5.751927e-17,2.696216e-18,1.685135e-17,-6.021549e-17,-1.177348e-16,1.258234e-17,1.617729e-17
std,1.000063e+00,1.000063e+00,1.000063e+00,1.000063e+00,1.000063e+00,1.000063e+00,1.000063e+00
min,-1.237843e+00,-1.976953e+00,-4.257448e-01,-3.401244e-01,-4.032398e-01,-3.680753e-01,-3.640256e-01
25%,-8.681525e-01,-6.438568e-01,-4.257448e-01,-3.401244e-01,-4.032398e-01,-3.680753e-01,-3.640256e-01
50%,-2.506900e-01,-1.527161e-01,-4.257448e-01,-3.401244e-01,-4.032398e-01,-3.680753e-01,-3.640256e-01
75%,7.738256e-01,6.190763e-01,-2.750822e-01,-2.941617e-01,-3.014848e-01,-2.789573e-01,-3.031277e-01
max,2.486596e+00,3.074780e+00,5.560223e+00,6.974923e+00,6.072972e+00,7.358692e+00,7.652572e+00


In [365]:
all_columns = df.columns
for column in all_columns:
  print(f'{column}:  {df[column].nunique()}')

HomePlanet:  3
CryoSleep:  2
Deck:  8
Num:  1808
Side:  2
Destination:  3
Age:  74
VIP:  2
RoomService:  1079
FoodCourt:  1184
ShoppingMall:  946
Spa:  1037
VRDeck:  1024
Transported:  2


In [366]:
cat_columns = [column for column in all_columns if column not in num_columns]

In [367]:
df.iloc[53]

HomePlanet              NaN
CryoSleep              True
Deck                      E
Num               -1.231944
Side                      S
Destination     TRAPPIST-1e
Age                0.338425
VIP                   False
RoomService       -0.425745
FoodCourt         -0.340124
ShoppingMall           -0.0
Spa               -0.368075
VRDeck            -0.364026
Transported            True
Name: 59, dtype: object

In [368]:
df

,HomePlanet,CryoSleep,Deck,Num,Side,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,B,-1.237843,P,TRAPPIST-1e,0.759402,False,-0.425745,-0.340124,-0.403240,-0.368075,-0.364026,False
1,Earth,False,F,-1.237843,S,TRAPPIST-1e,-0.293042,False,-0.131176,-0.327589,-0.321179,0.954242,-0.256845,True
4,Earth,False,F,-1.235877,S,TRAPPIST-1e,-0.854346,False,0.393103,-0.242628,0.092405,0.992780,-0.359154,True
5,Earth,False,F,-1.237843,P,PSO J318.5-22,1.110217,False,-0.425745,0.332604,-0.403240,0.332825,-0.364026,True
6,Earth,False,F,-1.233910,S,TRAPPIST-1e,-0.152716,False,-0.312241,1.803413,-0.393392,-0.368075,-0.364026,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8687,Europa,NaN,A,-1.047099,P,TRAPPIST-1e,0.127936,False,-0.425745,4.128016,-0.403240,-0.363258,0.439826,True
8689,Earth,True,G,1.709851,S,PSO J318.5-22,-0.714020,False,-0.425745,-0.340124,-0.403240,-0.368075,-0.364026,False
8690,Earth,False,G,1.711818,S,TRAPPIST-1e,-0.152716,False,-0.425745,-0.340124,5.741448,-0.365667,-0.364026,True
8691,Europa,False,E,-0.042247,S,55 Cancri e,0.268262,False,-0.425745,1.120935,-0.403240,0.482158,7.516160,False


In [369]:
df.isna().sum()

HomePlanet      193
CryoSleep       200
Deck            175
Num               0
Side            175
Destination     170
Age               0
VIP             186
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Transported       0
dtype: int64

In [370]:
for column in all_columns:
  print(f'{column}: {df[column].mode()[0]}')

HomePlanet: Earth
CryoSleep: False
Deck: F
Num: 0.0
Side: S
Destination: TRAPPIST-1e
Age: -0.2930420354926123
VIP: False
RoomService: -0.4257448040506429
FoodCourt: -0.3401244395631877
ShoppingMall: -0.4032397552219764
Spa: -0.3680752770631029
VRDeck: -0.36402560503659986
Transported: True


In [371]:
cat_columns = cat_columns[:-1]
print(cat_columns)

['HomePlanet', 'CryoSleep', 'Deck', 'Side', 'Destination', 'VIP']


Llenamos los valores faltantes categoricos con el valor más frecuente de la columna

In [372]:
for column in cat_columns:
  df[column] = df[column].fillna(df[column].mode()[0])

In [373]:
df.isnull().sum()

HomePlanet      0
CryoSleep       0
Deck            0
Num             0
Side            0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
dtype: int64

In [374]:
df.nunique()

HomePlanet         3
CryoSleep          2
Deck               8
Num             1808
Side               2
Destination        3
Age               74
VIP                2
RoomService     1079
FoodCourt       1184
ShoppingMall     946
Spa             1037
VRDeck          1024
Transported        2
dtype: int64

In [375]:
transported = df['Transported'].copy()
cryosleep = df['CryoSleep'].copy()
vip = df['VIP'].copy()

In [376]:
df.drop(['Transported', 'CryoSleep', 'VIP'], inplace=True, axis='columns')

In [377]:
df = pd.get_dummies(df)

In [378]:
df.head()

,Num,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,...,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Side_P,Side_S,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,-1.237843,0.759402,-0.425745,-0.340124,-0.403240,-0.368075,-0.364026,0,1,0,...,0,0,0,0,0,1,0,0,0,1
1,-1.237843,-0.293042,-0.131176,-0.327589,-0.321179,0.954242,-0.256845,1,0,0,...,0,0,1,0,0,0,1,0,0,1
4,-1.235877,-0.854346,0.393103,-0.242628,0.092405,0.992780,-0.359154,1,0,0,...,0,0,1,0,0,0,1,0,0,1
5,-1.237843,1.110217,-0.425745,0.332604,-0.403240,0.332825,-0.364026,1,0,0,...,0,0,1,0,0,1,0,0,1,0
6,-1.233910,-0.152716,-0.312241,1.803413,-0.393392,-0.368075,-0.364026,1,0,0,...,0,0,1,0,0,0,1,0,0,1


In [379]:
df['CryoSleep'] = cryosleep
df['VIP'] = vip

In [380]:
X = df[df.columns.tolist()]

In [381]:
y = transported

In [382]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [383]:
np.arange(0.01, 1, 0.1)

array([0.01, 0.11, 0.21, 0.31, 0.41, 0.51, 0.61, 0.71, 0.81, 0.91])

## Selección del modelo

Aquí creamos dos modelos, para después utilizar ambos para crear un nuevo clasificador, el cual se basará en utilizar ambos para realizar la predicción.

En este caso utilizamos XGB y KNeighbors

In [455]:
gbm_param_grid = {
    'learning_rate': [0.05],
    'colsample_bytree': [0.93],
    'n_estimators': [725],
    'max_depth': [5],
    'subsample': [0.96],
}

In [456]:
xg_cl=xgb.XGBClassifier(objective='binary:logistic' ,seed=42)

In [457]:
grid_acc=GridSearchCV(estimator=xg_cl, param_grid=gbm_param_grid, scoring='accuracy', cv=100, verbose=1)

In [458]:
grid_acc.fit(X_train, y_train)

Fitting 100 folds for each of 1 candidates, totalling 100 fits


GridSearchCV(cv=100,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=Non...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             param_grid={'colsample_bytree': [0.93], 'learning_rate': [0.05],
                         'max_depth': [5], 'n_estimators': [725],
                         'subsample': [0.96]},
             scoring='accuracy', verbose=1)

In [459]:
model = grid_acc.best_estimator_

In [460]:
params = grid_acc.best_params_

In [461]:
params

{'colsample_bytree': 0.93,
 'learning_rate': 0.05,
 'max_depth': 5,
 'n_estimators': 725,
 'subsample': 0.96}

In [462]:
preds = model.predict(X_test)

In [463]:
accuracy = accuracy_score(y_test, preds)
print('Voting Classifier: {:.8f}'.format(accuracy))

Voting Classifier: 0.80783818


In [541]:
knc_param_grid = {
    'n_neighbors': range(1,11),
    'leaf_size': range(1,10)
}

In [542]:
knc = KNeighborsClassifier()

In [543]:
grid_acc2=GridSearchCV(estimator=knc, param_grid=knc_param_grid, scoring='accuracy', cv=20, verbose=1)

In [544]:
grid_acc2.fit(X_train, y_train)

Fitting 20 folds for each of 90 candidates, totalling 1800 fits


GridSearchCV(cv=20, estimator=KNeighborsClassifier(),
             param_grid={'leaf_size': range(1, 10),
                         'n_neighbors': range(1, 11)},
             scoring='accuracy', verbose=1)

In [545]:
model2 = grid_acc2.best_estimator_

In [546]:
params = grid_acc2.best_params_
print(params)

{'leaf_size': 1, 'n_neighbors': 9}


In [547]:
preds2 = model2.predict(X_test)

In [548]:
accuracy_dtc = accuracy_score(y_test, preds2)
print('Voting Classifier: {:.8f}'.format(accuracy_dtc))

Voting Classifier: 0.77623262


In [549]:
classifiers = [('XGBClassifier', model), ('KNeighborsClassifier', model2)]

In [550]:
vc = VotingClassifier(estimators=classifiers)

In [551]:
vc.fit(X_train, y_train)

VotingClassifier(estimators=[('XGBClassifier',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.93, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            l...ng_rate=0.05, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=5,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=725, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...)),
                             ('KNeighborsClassifier',
                              KNeighborsClassifier(leaf_size=1,
                                                   n_neighbors=9))])

In [552]:
y_pred = vc.predict(X_test)

In [553]:
accuracy_final = accuracy_score(y_test, y_pred)
print('Voting Classifier: {:.8f}'.format(accuracy_final))

Voting Classifier: 0.79266751


## Submission

Aquí ya procedemos a cargar el test, pasamos por el mismo preprocesamiento de el dataset original omitiendo la eliminación de outliers. Procedemos a crear nuestros predicts finales los cuales subiremos utilizando el modelo generado a partir de los dos modelos iniciales.

In [554]:
df_test = pd.read_csv('test titanic.csv')

In [555]:
df_test[['Deck', 'Num', 'Side']] = df_test.Cabin.str.split('/', expand=True)

In [556]:
df_test = df_test[['PassengerId',	'HomePlanet',	'CryoSleep', 'Deck', 'Num', 'Side', 'Cabin', 'Destination',	'Age',	'VIP',
         'RoomService',	'FoodCourt',	'ShoppingMall',	'Spa',	'VRDeck',	'Name']]

In [557]:
df_test['Num'] = pd.to_numeric(df_test['Num'], errors='coerce')

In [558]:
passenger = df_test[['PassengerId']]

In [559]:
df_test.drop(['PassengerId', 'Cabin', 'Name'], inplace=True, axis='columns')

In [560]:
num_imput = SimpleImputer(missing_values=np.NaN, strategy='mean')
df_test[num_columns] = num_imput.fit_transform(df_test[num_columns])

In [561]:
df_test[num_columns] = StandardScaler().fit_transform(df_test[num_columns])

In [562]:
for column in cat_columns:
  df_test[column] = df_test[column].fillna(df_test[column].mode()[0])

In [563]:
cryosleep = df_test['CryoSleep'].copy()
vip = df_test['VIP'].copy()

In [564]:
df_test.drop(['CryoSleep', 'VIP'], inplace=True, axis='columns')

In [565]:
df_test = pd.get_dummies(df_test)

In [566]:
df_test['CryoSleep'] = cryosleep
df_test['VIP'] = vip

In [567]:
final_preds= vc.predict(df_test)

In [568]:
df_test['PassengerId'] = passenger

In [569]:
df_test['Transported'] = final_preds

In [570]:
submission = df_test[['PassengerId', 'Transported']]

In [571]:
submission['Transported'] = submission['Transported'].astype(bool)

<ipython-input-571-7ac02d6b6392>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['Transported'] = submission['Transported'].astype(bool)


In [572]:
submission

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,False


In [573]:
submission.to_csv('submission.csv', index=False)